<a href="https://colab.research.google.com/github/shiehn/rune_notebooks/blob/main/openapi_game_engine_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os


# Check if ffmpeg is installed, if not then install it
ffmpeg_installed = !command -v ffmpeg
if not ffmpeg_installed:
    !apt-get install ffmpeg

#Check if runes-client is installed, if not then install it
try:
    import runes_client
except ImportError:
    !pip install runes-client --upgrade
    import runes_client


import runes_client.core as runes
from runes_client import RunesFilePath

placeholder_txt = "Enter the token generated by the Crucible plugin"
RUNES_TOKEN = "88140973-6320-4e4c-b903-dbb8ecb13e42" #@param {type:"string"}
runes_token = RUNES_TOKEN

dn_cli_token = os.getenv('DN_CLI_TOKEN', None)
if dn_cli_token is not None and dn_cli_token != "":
  runes_token = dn_cli_token
else:
  if runes_token is None or runes_token == "" or runes_token == placeholder_txt:
      print("ERROR: The token provided is not valid.")
      exit()

OPENAI_API_KEY = "xxx" #@param {type:"string"}
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

if os.getenv('OPENAI_API_KEY', None) is None or os.getenv('OPENAI_API_KEY', None) == "":
  print("ERROR: The OpenAI API key is not valid.")
  exit()

# There are two separate function you need to register.
# 1. The imports.  This is where you load any libraries you need to use in your RUNE.  This is important so that the RUNE is aware when dependencies are loaded and its ready to be called from the plugin.
# 2. The method.  This is the primary function of the RUNE.  This is where you write the code that will be executed when the RUNE is triggered.

# Name the function anything you like.  This is the imports function.  This is where you load any libraries you need to use in your RUNE.
async def register_imports():

    import requests
    import tempfile
    import uuid

    !pip install openai

    from openai import OpenAI
    global client
    client = OpenAI()

    runes.make_imports_global([requests, tempfile, uuid])


# Now register the imports function with the discovery server.
runes.register_imports(register_imports)

async def runes_func(image_description: str):
    try:
        response = client.images.generate(
          model="dall-e-3",
          prompt=image_description,
          size="1024x1024",
          quality="standard",
          n=1,
        )

        image_url = response.data[0].url

        # Create a temporary directory
        tmp_dir = tempfile.mkdtemp()

        # Get the file extension from the URL
        #_, file_extension = os.path.splitext(image_url)
        file_extension = ".png"

        # Create a random UUID for the new file name
        file_name = str(uuid.uuid4()) + file_extension

        # Construct the path to the new file in the temporary directory
        file_path = os.path.join(tmp_dir, file_name)

        # Download the image and save it to the new file
        response = requests.get(image_url, stream=True)
        with open(file_path, 'wb') as f:
            f.write(response.content)

        # after executing your custom code you send data back to the plugin like so ..
        await runes.output().add_file(file_path)
        #await runes.output().add_message(response)
        await runes.output().add_log(image_url)
        await runes.output().send()

        return True
    except Exception as e:
        await runes.output().add_error(f"Error in arbitrary_method: {e}")

        return False


runes.set_token(token=runes_token)
runes.set_name("OpenAPI Image Engine")
runes.set_description("Game image engine powered by OpenAI")
runes.set_type("image_engine")
runes.register_method(runes_func)

runes.connect_to_server()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 12.3 MB/s eta 0:00:00
CREATING CONTRACT {'method_name': 'runes_func', 'params': [{'name': 'image_description', 'type': 'str', 'default_value': '', 'ui_component': None}], 'author': 'Default Author', 'name': 'OpenAPI Image Engine', 'description': 'Game image engine powered by OpenAI', 'version': '0.0.0'}
PENDING REQUESTS: []
Successfully created compute contract. Response: {"id":"d25dfa67-f1de-5441-85f2-0715703f0423","data":{"method_name":"runes_func","params":[{"name":"image_description","type":"str","default_value":"","ui_component":null}],"author":"Default Author","name":"OpenAPI Image Engine","description":"Game image engine powered by OpenAI","version":"0.0.0"},"created_at":"2024-04-25T05:08:51","updated_at":"2024-04-25T05:08:51"}
Heartbeat successful.
ADD_MAPPIN

KeyboardInterrupt: 